<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/dev/examples/AI_Generated_Voiceovers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai videodb

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"
os.environ["ELEVEN_LABS_API_KEY"] = "YOUR_ELEVEN_LABS_KEY"
os.environ["VIDEO_DB_API_KEY"] = "YOUR_VIDEO_DB_KEY"

In [7]:
from videodb import connect, play_stream

# Connect to VideoDB using your API key
conn = connect()

In [8]:
video_url = 'https://youtu.be/RcRjY5kzia8'
video = conn.upload(url=video_url)

In [10]:
video.index_scenes()

In [11]:
scenes = video.get_scenes()

In [30]:
import openai

client = openai.OpenAI()

script_prompt = "Here's the data from a scene index for a video about the underwater world. Study this and then generate a synced script based on the description below. Make sure the script is in the language, voice and style of Sir David Attenborough"

def generate_voiceover_script(scenes, script_prompt):
    full_prompt = script_prompt + "\n\n"
    for scene in scenes:
        full_prompt += f"- {scene}\n"

    
    res = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{ "role": "system",
        "content": full_prompt 
    }],
    )
    
    return res

openai_res = generate_voiceover_script(scenes=scenes, script_prompt=script_prompt)
voiceover_script = openai_res.choices[0].message.content

# if you have eleven labs paid plan remove this
voiceover_script = voiceover_script[:2500]

In [31]:
import requests


CHUNK_SIZE = 1024
audio_file = "audio.mp3"
voice_id = "jkgil0y3OJaiiupjsrCB"
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"

headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}

payload = {
    "model_id": "eleven_monolingual_v1",
    "text": voiceover_script,
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}

elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)
with open(audio_file, 'wb') as f:
    for chunk in elevenlabs_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

In [32]:
audio = conn.upload(file_path=audio_file)

In [33]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

# Create a timeline object to manage video assets and overlays
timeline = Timeline(conn)

# Add the video asset to the timeline for playback
video_asset = VideoAsset(asset_id=video.id)
timeline.add_inline(asset=video_asset)

# Add the audio asset to the timeline for playback
audio_asset = AudioAsset(asset_id=audio.id)  # Assuming 'audio' holds the voiceover audio asset
timeline.add_overlay(start=0, asset=audio_asset)

stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/5e39f02c-3886-4cbc-a193-b6f672acbaab.m3u8'